In [55]:
import numpy as np
from sklearn.metrics import pairwise_distances
from scipy.optimize import fsolve, newton_krylov, broyden1, leastsq
from scipy.optimize import approx_fprime
from scipy.integrate import solve_ivp


def psi_function(row_1, row_2):
    psi_i = row_1[0]
    psi_j = row_2[0]
    if psi_i + psi_j == 0:
        return 0
    return (psi_i - psi_j) *np.abs(psi_i - psi_j)/(psi_i + psi_j)


class Simulator:
    def __init__(self, p, tstep = 0.1) -> None:
        """ ANY changes here must also go to reset method below"""
        self.secant_used = False
        self.t = 0
        self.tstep = tstep
        self.p = p
        # calculate T
        self.N = self.p['coords'].shape[0]
        self._calculate_T()
        self.u = p['u0']
        self.q = p['q0']
        self.num_psis = p['mu'].shape[1]
        # init x with only the S population
        self.x = np.zeros([self.N,self.num_psis])
        self.x[:,0] = self.p['S0']
        self.dead_tracker = [np.zeros([self.N])]
        self.source_sum = np.zeros(self.N)

        # one time calc
        self.T_expanded = np.tile(self.p['T'][...,np.newaxis], [1,1,self.num_psis])
        self.P_expanded = np.tile(self.p['P'][...,np.newaxis], [1,1,self.num_psis])
        self.M_expanded = np.tile(self.p['M'][np.newaxis,...],[self.N,1,1])

    def reset(self, p = None, tstep = None):
        self.t = 0
        self.tstep = tstep
        self.p = p
        # calculate T
        self.N = self.p['coords'].shape[0]
        self._calculate_T()
        self.u = p['u0']
        self.q = p['q0']
        self.num_psis = p['mu'].shape[1]
        self.x = np.zeros([self.N, self.num_psis])
        # init x with only the S population
        self.x = np.zeros([self.N,self.num_psis])
        self.x[:,0] = self.p['S0']
        self.dead_tracker = [np.zeros([self.N])]

        # one time calc
        self.T_expanded = np.tile(self.p['T'][...,np.newaxis], [1,1,self.num_psis])
        self.P_expanded = np.tile(self.p['P'][...,np.newaxis], [1,1,self.num_psis])
        self.M_expanded = np.tile(self.p['M'][np.newaxis,...],[self.N,1,1])

    def _calculate_T(self):
        # T is an N by N matrix
        coords = self.p['coords'] # N x 2
        deltas = pairwise_distances(coords)
        exp_deltas = np.exp(-1 * deltas)
        self.p['T'] = exp_deltas / ((np.sum(exp_deltas) - self.N)/2)
        np.fill_diagonal(self.p['T'], 0)


    def _calculate_tau_matrix(self):
        # tau is N x 4 (SIRD)
        # self.x.shape is N x 4
        psi_matrix = np.array([pairwise_distances(self.x[:,i:i+1], metric = psi_function) for i in range(self.num_psis)])
        psi_matrix = np.transpose(psi_matrix, (1, 2, 0))
        for i in range(self.num_psis):
          psi_matrix[(*np.tril_indices(self.N, -1),i)] *= -1
        result = psi_matrix * self.T_expanded * self.P_expanded * self.M_expanded
        # result = np.floor(psi_matrix * self.T_expanded * self.P_expanded * self.M_expanded)
        return result

    def u_sources(self):
        if self.p['mode'] == 'exp':
            self.u = np.apply_along_axis(lambda x :np.floor(x[0]*np.exp(-1*x[1]*self.t)) , 1, np.vstack([self.p['u0'], self.p['gamma']]).T)
        elif self.p['mode'] == 'lin':
            self.u = np.apply_along_axis(lambda x : np.max([0,np.floor(x[0] - x[1]*self.t)]) , 1, np.vstack([self.p['u0'], self.p['zeta']]).T)

    def q_intra_sources(self):
        if self.p['mode'] == 'exp':
            self.q = np.apply_along_axis(lambda x :np.floor(x[0]*np.exp(-1*x[1]*self.t)) , 1, np.vstack([self.p['q0'], self.p['gamma']]).T)
        elif self.p['mode'] == 'lin':
            self.q = np.apply_along_axis(lambda x : np.max([0,np.floor(x[0] - x[1]*self.t)]) , 1, np.vstack([self.p['q0'], self.p['zeta']]).T)

    def eval_f(self, custom_x = None, active_step = False, dt = None):
        # extract params
        if custom_x is None:
          x = self.x
        else:
          x = custom_x

        if active_step:
            assert dt is not None

        if x.shape[0] == x.size:
            x = np.reshape(x, (self.N, self.num_psis))
        p = self.p
        u = self.u
        q = self.q
        N = p['N']
        mu = p['mu']
        num_psis = self.num_psis
        v = p['v']
        alpha = p['alpha']
        beta = p['beta']
        kappa = p['kappa']
        # init f matrix
        f = np.zeros([N,num_psis])
        # get tau
        tau = self._calculate_tau_matrix()
        # sum tau over AXIS = 1
        tau = np.sum(tau, axis=1) # N x num_psi(4)

        print("Shape of q:", q.shape)
        print("Shape of u:", u.shape)
        print("Shape of x[:,0]:", x[:,0].shape)
        
        q = np.apply_along_axis(lambda x : x[2] if x[0] - x[1] > x[2] else x[0], 1, np.vstack([q,u, x[:,0]]).T)

        # number of people per node
        sigma = np.sum(x, axis = 1)# + self.dead_tracker[-1]
        # ensure no sigma is zero
        sigma[np.where(sigma == 0)] = 1e-5
        mu[np.where(mu == 0)] = 1e-5
        # evaluate f

        vaccination_rate = 0.1*(v * (x[:,0]**2)/ (sigma))
        max_vaccination_rate = np.inf  # Set this to a reasonable value
        vaccination_rate = np.minimum(vaccination_rate, max_vaccination_rate)

        # POSSIBLE FIX: do Guarding of u - q AFTER calculating the other terms
        f[:,0] = (-alpha * x[:,1] * x[:,0] / sigma) - mu[:,0] * x[:,0] - q - tau[:,0] - vaccination_rate + u
        f[:,1] = (alpha * x[:,1] * x[:,0] / sigma) - (mu[:,1] + beta + kappa) * x[:,1] + q - tau[:,1]
        f[:,2] = beta * x[:,1] - mu[:,2] * x[:,2] +vaccination_rate - tau[:,2]

        if active_step:
            self.dead_tracker.append(self.dead_tracker[-1] + dt * (kappa * x[:,1] + mu[:,0] * x[:,0] + mu[:,1] * x[:,1]  + mu[:,2] * x[:,2]))
            self.source_sum += dt * u
            self.u_sources()
            self.q_intra_sources()

        return f
    
    def solve_steady_state(self, guess = None):
        if guess is None:
            guess = np.reshape(self.x, (self.N * self.num_psis))
        else:
            guess = np.reshape(guess, (self.N * self.num_psis))
        root = fsolve(self.eval_f, guess, True, xtol=1)
        return root
    
    def Jac_test(self): 
        def _eval_f_wrapper(x_flat):
            x_reshaped = x_flat.reshape(self.x.shape)
            f_val = self.eval_f(x_reshaped)
            return f_val.flatten()

        x = self.x.flatten()
        epsilon = np.sqrt(np.finfo(float).eps)
        jacobian = approx_fprime(x, _eval_f_wrapper, epsilon)
        return jacobian.reshape(self.x.shape[0] * self.x.shape[1], -1)


    def FiniteDifference_JacobianMatrix(self, x_custom=None):
        if x_custom is None:
            x_col = self.x.flatten().reshape(-1, 1)
        else:
            x_col = x_custom.flatten().reshape(-1, 1)

        total_states = len(x_col)
        Jf = np.zeros((total_states, total_states))

        # Adaptive epsilon based on the norm of x_col
        norm_x = np.linalg.norm(x_col, np.inf)
        base_epsilon = 2 * np.sqrt(np.finfo(float).eps) * np.sqrt(1 + norm_x)

        for i in range(total_states):
            epsilon_i = base_epsilon * (1 + abs(x_col[i, 0]))
            e_i = np.zeros((total_states, 1))
            e_i[i, 0] = epsilon_i

            x_perturbed = (x_col + e_i).reshape(self.x.shape)
            delta_f = (self.eval_f(x_perturbed) - self.eval_f(x_perturbed - e_i.reshape(self.x.shape))).flatten()

            Jf[:, i] = delta_f / epsilon_i

        return Jf

    def forward_euler(self, tspan, dt, return_tau = False):
        """
        Uses Forward Euler to simulate states model dx/dt=f(x,p,u)
        starting from state vector x_start at time t_start
        until time t_stop, with time intervals timestep.

        Parameters:
        - f: function that evaluates f(x,p,u)
        - x: initial state vector
        - tspan: tuple of (start, end) time
        - dt: time step for the integration

        Returns:
        - t: array of time points
        - y: array of state vectors at each time point
        """

        t_start = tspan[0]
        t_stop = tspan[1]

        num_steps = len(np.arange(t_start, t_stop, dt))
        t = np.zeros(num_steps + 1)

        # Initialize the state trajectory array
        y = np.zeros((num_steps + 1,) + self.x.shape)
        y[0] = self.x

        self.t = t_start
        t[0] = self.t

        taus = []

        for n in range(num_steps):
            #if np.any(self.x <0):
              #break
            dt = min(dt, (t_stop - t[n]))
            f_val = self.eval_f(dt=dt, active_step=True)
            self.x += dt * f_val
            y[n+1] = self.x
            self.t += dt
            t[n+1] = self.t
            # print(f"Time step {t[n+1]}: Sum of x = {np.sum(self.x)}")
            #print(f"Time step {t[n+1]}: Sum of x = {np.sum(self.x) + np.sum(self.dead_tracker[-1])}")
            # print(f"Time step {t[n+1]}: New Source Sum (per node/total) = {self.source_sum}, {np.sum(self.source_sum)}")
            # if self.t % 10 == 0:
            #     print("Time: ", self.t)
            if return_tau:
              tau = self._calculate_tau_matrix()
              taus.append(tau)
        if return_tau:
          return t, y, taus

        return t, y

    def trapezoidal_equation(self, x_new_flat, x_old_flat, dt):
        """
        Calculate the trapezoidal rule result.

        Parameters:
        - x_new_flat: flattened state at the next time step
        - x_old_flat: flattened state at the current time step
        - dt: time step size

        Returns:
        - result: result of the trapezoidal rule
        """
        # Reshape the flattened states back to their original multi-dimensional structure
        x_new = x_new_flat.reshape(self.x.shape)
        x_old = x_old_flat.reshape(self.x.shape)

        # Calculate f for current and next time steps
        f_current = self.eval_f(custom_x=x_old).flatten()  # Flatten the result
        f_next = self.eval_f(custom_x=x_new).flatten()  # Flatten the result

        # Compute the trapezoidal rule result
        result = x_new_flat - x_old_flat - (dt / 2) * (f_current + f_next)
        return result

    from scipy.optimize import approx_fprime

########################## GCR CODE ####################################

    def trapezoidal_method_GCRHadpt(self, tspan, initial_dt, tolerance, max_dt=1, max_iters_newton=50):
        t_start, t_stop = tspan
        max_estimated_steps = int((t_stop - t_start) / initial_dt) + 1
        t = np.zeros(max_estimated_steps)
        y = np.zeros((max_estimated_steps,) + self.x.shape)
        y[0] = self.x.copy()
        t[0] = t_start

        self.t = t_start
        dt = initial_dt
        n = 0

        while self.t < t_stop and n < max_estimated_steps - 1:
            x_current = self.x.copy()
            f_current = self.eval_f(x_current, active_step=True, dt=dt)

            def trapezoidal_implicit_function(x_next_flat):
                x_next = x_next_flat.reshape(self.x.shape)
                f_next = self.eval_f(x_next)
                return (x_next - x_current - (dt / 2) * (f_current + f_next)).flatten()

            x_next_flat_guess = x_current.flatten()
            x_next_flat, converged = self.HybridSolver_GCR(
                x_next_flat_guess,
                trapezoidal_implicit_function,
                max_iters_newton = max_iters_newton
            )

            if not converged:
                print(f"Newton-GCR method failed to converge at time step {n + 1}. Decreasing dt.")
                dt = max(initial_dt, dt / 1.5)
                continue

            self.x = x_next_flat.reshape(self.x.shape)
            y[n + 1] = self.x.copy()

            # Calculate time remaining and adjust dt
            time_remaining = t_stop - self.t
            dt = min(dt, time_remaining)  # Ensure dt does not exceed time remaining

            self.t += dt
            t[n + 1] = self.t
            n += 1

            
            change = np.linalg.norm(self.x - x_current, np.inf)
            gradual_reduction_threshold = 0 * (t_stop - t_start)  

            if self.t >= t_stop - gradual_reduction_threshold:
                dt = max(initial_dt, dt / 1.1)  
            elif change < tolerance / 2 and dt < max_dt:
                dt = min(dt * 1.1, max_dt, time_remaining)  
            #elif change > tolerance:
                #dt = max(initial_dt, dt / 1.5)  # Decrease dt if change is large

            print(f"Time step {n + 1}: dt = {dt}, change = {change}")

        t = t[:n + 1]
        y = y[:n + 1]

        return t, y
    

    def trapezoidal_method_GCRadpt(self, tspan, initial_dt, tolerance, max_dt=1, max_iters_newton=50):
        t_start, t_stop = tspan
        max_estimated_steps = int((t_stop - t_start) / initial_dt) + 1
        t = np.zeros(max_estimated_steps)
        y = np.zeros((max_estimated_steps,) + self.x.shape)
        y[0] = self.x.copy()
        t[0] = t_start

        self.t = t_start
        dt = initial_dt  # Start with the minimum dt
        n = 0

        while self.t < t_stop and n < max_estimated_steps - 1:
            x_current = self.x.copy()
            f_current = self.eval_f(x_current, active_step=True, dt=dt)

            def trapezoidal_implicit_function(x_next_flat):
                x_next = x_next_flat.reshape(self.x.shape)
                f_next = self.eval_f(x_next)
                return (x_next - x_current - (dt / 2) * (f_current + f_next)).flatten()

            x_next_flat_guess = x_current.flatten()
            x_next_flat, converged = self.NewtonNd_GCR(
                x_next_flat_guess,
                trapezoidal_implicit_function,
                tol_f=tolerance,
                max_iter=max_iters_newton
            )

            if not converged:
                print(f"Newton-GCR method failed to converge at time step {n + 1}. Decreasing dt.")
                dt = max(initial_dt, dt / 1.1)
                continue

            self.x = x_next_flat.reshape(self.x.shape)
            y[n + 1] = self.x.copy()

            time_remaining = t_stop - self.t
            dt = min(dt, time_remaining)  

            self.t += dt
            t[n + 1] = self.t
            n += 1

            # Adjust dt based on solution change and phase of simulation
            change = np.linalg.norm(self.x - x_current, np.inf)
            gradual_reduction_threshold = 0.005 * (t_stop - t_start)  

            if self.t >= t_stop - gradual_reduction_threshold:
                dt = max(initial_dt, dt / 1.25)  
            elif change < tolerance / 2 and dt < max_dt:
                dt = min(dt * 1.25, max_dt, time_remaining)  
            #elif change > tolerance:
                #dt = max(initial_dt, dt / 1.5)  # Decrease dt if change is large

            print(f"Time step {n + 1}: dt = {dt}, change = {change}")

        # Trim the arrays to the actual number of steps taken
        t = t[:n + 1]
        y = y[:n + 1]

        return t, y

    def trapezoidal_method_GCR(self, tspan, dt):
            t_start, t_stop = tspan
            num_steps = int((t_stop - t_start) / dt)
            t = np.zeros(num_steps + 1)
            y = np.zeros((num_steps + 1,) + self.x.shape)
            y[0] = self.x

            self.t = t_start
            t[0] = self.t
            for n in range(num_steps):
                x_current = self.x
                f_current = self.eval_f(x_current, active_step=True, dt=dt)
                def trapezoidal_implicit_function(x_next_flat, f_current):
                    x_next = x_next_flat.reshape(self.x.shape)
                    f_next = self.eval_f(x_next)
                    return (x_next - x_current - (dt / 2) * (f_current + f_next)).flatten()

                x_next_flat_guess = x_current.flatten()
                # Use newtonNd_GCR for solving the implicit equation
                x_next_flat, converged= self.NewtonNd_GCR(
                    x_next_flat_guess,
                    trapezoidal_implicit_function,
                    tol_f=1e-9,  
                    max_iter=20,  
                )

                if not converged:
                    print(f"Newton-GCR method failed to converge at time step {n + 1}")

                self.x = x_next_flat.reshape(self.x.shape)
                y[n + 1] = self.x

                self.t += dt
                t[n + 1] = self.t

            return t, y

    def tgcr_matrixfree(self, xk, b, eps, tolrGCR, MaxItersGCR):
            x = np.zeros_like(b)
            r = b.copy()
            r_norms = [np.linalg.norm(r, 2)]
            P_matrix = np.zeros((len(b), MaxItersGCR))
            gcr_converged = False  

            for k in range(MaxItersGCR):
                P_matrix[:, k] = r / (np.linalg.norm(r) + 1e-15)  
                Ap = (self.eval_f(xk + eps * P_matrix[:, k]) - self.eval_f(xk)) / eps 
                Ap = Ap.flatten()

                # Orthogonalize Ap against previous directions in P_matrix
                for j in range(k):
                    Ap -= np.dot(Ap, P_matrix[:, j]) * P_matrix[:, j]

                Ap_norm = np.linalg.norm(Ap)
                if Ap_norm < 1e-15:  
                    break

                Ap /= Ap_norm
                P_matrix[:, k] = Ap

                
                alpha = np.dot(r, Ap)
                x += alpha * Ap
                r -= alpha * Ap
                new_r_norm = np.linalg.norm(r)
                r_norms.append(new_r_norm)

                
                if new_r_norm <= tolrGCR * r_norms[0]:
                    gcr_converged = True
                    break

            return x, gcr_converged, r_norms


    def NewtonNd_GCR(self, x0_flat, eval_f, tol_f=1e-9, max_iter=1000, custom_J = None, return_res = False):
            k = 0
            x_flat = x0_flat
            f = eval_f(x_flat)
            err_f = np.linalg.norm(f, np.inf)
            tolrGCR = 1e-8
            MaxItersGCR = 10000
            eps = 1e-4; 
            eps_a = np.sqrt(np.finfo(float).eps * eps)

            while k < max_iter:

                delta_x, _, _ = self.tgcr_matrixfree(x_flat, -f.flatten(), eps_a, tolrGCR, MaxItersGCR)
                
                step_size = 1 # Adjust based on your problem scale

                x_new_flat = x_flat + step_size * delta_x
                x_flat = x_new_flat
                k += 1

                f = eval_f(x_flat)
                err_f = np.linalg.norm(f, np.inf)

                #print(f"Iteration {k}: Residual norm = {err_f}")

                if err_f <= tol_f:
                    #print(f"Newton's method converged in {k} iterations with residual norm: {err_f}")
                    if return_res:
                        return x_flat, True, err_f
                    else:
                        return x_flat, True

            print("Newton did NOT converge! Maximum Number of Iterations reached")
            if return_res:
                return x_flat, True, err_f
            else:
                return x_flat, True
        
    def HybridSolver_GCR(self, x0_flat, func, max_iters_newton):
        if not self.secant_used:
            # Use the Secant method only if it hasn't been used before
            x1_flat = x0_flat + np.random.randn(*x0_flat.shape) * 1e-4
            x_initial_guess, secant_converged = self.secant_method_solver(x0_flat, x1_flat, func)
            self.secant_used = True
        else:
            # If Secant has been used before, just use the current guess
            x_initial_guess = x0_flat

        # Proceed with Newton-GCR method
        x_newton_GCR, newton_GCR_converged, final_residual_norm = self.NewtonNd_GCR(
            x_initial_guess,
            func,
            tol_f=1e-9,
            max_iter=max_iters_newton,
            return_res=True
        )

        return x_newton_GCR, newton_GCR_converged

################### NEWTON NORMAL TRAP CODE ####################################

    def trapezoidal_method(self, tspan, dt):
        t_start, t_stop = tspan
        num_steps = int((t_stop - t_start) / dt)
        t = np.zeros(num_steps + 1)
        y = np.zeros((num_steps + 1,) + self.x.shape)
        y[0] = self.x

        self.t = t_start
        t[0] = self.t

        for n in range(num_steps):
            x_current = self.x
            f_current = self.eval_f(x_current, active_step=True, dt=dt)

            def trapezoidal_implicit_function(x_next_flat):
                x_next = x_next_flat.reshape(self.x.shape)
                f_next = self.eval_f(x_next)
                return (x_next - x_current - (dt / 2) * (f_current + f_next)).flatten()
            
            def J_trapezoidal_implicit_function(x_next_flat):
                x_next = x_next_flat.reshape(self.x.shape)
                return np.eye(x_next.size) - dt/2 * self.FiniteDifference_JacobianMatrix(x_next)        

            x_next_flat_guess = x_current.flatten()
            x_next_flat, converged = self.NewtonNd(x_next_flat_guess, trapezoidal_implicit_function,
                                                   custom_J = J_trapezoidal_implicit_function)

            if not converged:
                print(f"Newton's method failed to converge at time step {n + 1}")

            self.x = x_next_flat.reshape(self.x.shape)
            y[n + 1] = self.x

            self.t += dt
            t[n + 1] = self.t

        return t, y

    def NewtonNd(self, x0_flat, eval_f, tol_f=1e-4, max_iter=1000, custom_J = None, return_res = False):
        k = 0
        x_flat = x0_flat
        f = eval_f(x_flat)
        err_f = np.linalg.norm(f, np.inf)

        while k < max_iter:
            if custom_J is None:
                Jf = self.FiniteDifference_JacobianMatrix(x_flat)
            else:
                Jf = custom_J(x_flat)

            delta_x = np.linalg.solve(Jf, -f.flatten())

            # Fixed step size
            step_size = 1 # Adjust based on your problem scale

            x_new_flat = x_flat + step_size * delta_x
            x_flat = x_new_flat
            k += 1

            f = eval_f(x_flat)
            err_f = np.linalg.norm(f, np.inf)

            #print(f"Iteration {k}: Residual norm = {err_f}")

            if err_f <= tol_f:
                #print(f"Newton's method converged in {k} iterations with residual norm: {err_f}")
                if return_res:
                    return x_flat, True, err_f
                else:
                    return x_flat, True

        print("Newton did NOT converge! Maximum Number of Iterations reached")
        if return_res:
            return x_flat, True, err_f
        else:
            return x_flat, True

    def secant_method_solver(self, x0_flat, x1_flat, func, epsilon=1e-2, max_iters=1000):
        for _ in range(max_iters):
            f_x0 = func(x0_flat)
            f_x1 = func(x1_flat)

            if np.linalg.norm(f_x1 - f_x0) < 1e-8:
                print("Denominator too small. Secant method failed.")
                return x1_flat, False

            x2_flat = x1_flat - f_x1 * (x1_flat - x0_flat) / (f_x1 - f_x0)
            if np.linalg.norm(x2_flat - x1_flat) < epsilon:
                return x2_flat, True  # Converged

            x0_flat, x1_flat = x1_flat, x2_flat

        print("Secant method did not converge.")
        return x1_flat, False

    def HybridSolver_s(self, x0_flat, func, J_func, convergence_threshold, max_iters_newton):
        # Always use the Secant method to generate an initial guess
        x1_flat = x0_flat + np.random.randn(*x0_flat.shape) * 1e-2
        x_initial_guess, secant_converged = self.secant_method_solver(x0_flat, x1_flat, func)

        # Proceed with Newton's method using the initial guess from the Secant method
        x_newton, newton_converged, final_residual_norm = self.NewtonNd(x_initial_guess, func, max_iters_newton, custom_J=J_func, return_res=True)

        return x_newton, newton_converged


    def HybridSolver(self, x0_flat, func, J_func, convergence_threshold, max_iters_newton):
        if not self.secant_used:
            # Use the Secant method only if it hasn't been used before
            x1_flat = x0_flat + np.random.randn(*x0_flat.shape) * 1e-4
            x_initial_guess, secant_converged = self.secant_method_solver(x0_flat, x1_flat, func)
            self.secant_used = True  
            #print("Secant method used for initial guess.")
        else:
            # If Secant has been used before, just use the current guess
            x_initial_guess = x0_flat
            #print("Using current guess for Newton's method.")

        # Proceed with Newton's method
        x_newton, newton_converged, final_residual_norm = self.NewtonNd(x_initial_guess, func, max_iters_newton, custom_J=J_func, return_res = True)

        #if newton_converged:
            #print(f"Newton's method converged with final residual norm: {final_residual_norm}")
        #else:
            #print("Newton's method did not converge.")

        return x_newton, newton_converged
    
    def trapezoidal_method_sec(self, tspan, dt):
        t_start, t_stop = tspan
        num_steps = int((t_stop - t_start) / dt)
        t = np.zeros(num_steps + 1)
        y = np.zeros((num_steps + 1,) + self.x.shape)
        y[0] = self.x

        self.t = t_start
        t[0] = self.t

        for n in range(num_steps):
            x_current = self.x
            f_current = self.eval_f(x_current, active_step=True, dt=dt)

            def trapezoidal_implicit_function(x_next_flat):
                x_next = x_next_flat.reshape(self.x.shape)
                f_next = self.eval_f(x_next)
                return (x_next - x_current - (dt / 2) * (f_current + f_next)).flatten()
            

            x_next_flat_guess = x_current.flatten()
            x_next_flat_guess_prev = x_next_flat_guess + 1e-4  # Slightly perturbed guess

            x_next_flat, converged = self.secant_method_solver(
                x_next_flat_guess_prev, x_next_flat_guess, trapezoidal_implicit_function)

            if not converged:
                print(f"Secant method failed to converge at time step {n + 1}")

            self.x = x_next_flat.reshape(self.x.shape)
            y[n + 1] = self.x

            self.t += dt
            t[n + 1] = self.t

        return t, y
    
    def trapezoidal_method_hybrid(self, tspan, dt, convergence_threshold=1e-1, max_iters_newton=50):
        t_start, t_stop = tspan
        num_steps = int((t_stop - t_start) / dt)
        t = np.zeros(num_steps + 1)
        y = np.zeros((num_steps + 1,) + self.x.shape)
        y[0] = self.x

        self.t = t_start
        t[0] = self.t

        for n in range(num_steps):
            x_current = self.x
            f_current = self.eval_f(x_current, active_step=True, dt=dt)

            def trapezoidal_implicit_function(x_next_flat):
                x_next = x_next_flat.reshape(self.x.shape)
                f_next = self.eval_f(x_next)
                return (x_next - x_current - (dt / 2) * (f_current + f_next)).flatten()

            def J_trapezoidal_implicit_function(x_next_flat):
                x_next = x_next_flat.reshape(self.x.shape)
                return np.eye(x_next.size) - dt/2 * self.FiniteDifference_JacobianMatrix(x_next)   

            x_next_flat_guess = x_current.flatten()

            # Using Hybrid Solver
            x_next_flat, converged = self.HybridSolver(x_next_flat_guess, trapezoidal_implicit_function, J_trapezoidal_implicit_function, convergence_threshold, max_iters_newton)

            if not converged:
                print(f"Solver failed to converge at time step {n + 1}")

            self.x = x_next_flat.reshape(self.x.shape)
            y[n + 1] = self.x

            self.t += dt
            t[n + 1] = self.t

        return t, y
    
    def trapezoidal_method_adapt(self, tspan, initial_dt, tolerance, convergence_threshold=1e-1, max_dt=1, max_iters_newton=50):
        t_start, t_stop = tspan
        max_estimated_steps = int((t_stop - t_start) / initial_dt) + 1  # Estimate based on initial_dt for maximum steps
        t = np.zeros(max_estimated_steps)
        y = np.zeros((max_estimated_steps,) + self.x.shape)
        y[0] = self.x.copy()
        t[0] = t_start

        self.t = t_start
        dt = initial_dt  # Start with initial_dt
        n = 0

        while self.t < t_stop and n < max_estimated_steps - 1:
            x_current = self.x.copy()
            f_current = self.eval_f(x_current, active_step=True, dt=dt)
            
            def trapezoidal_implicit_function(x_next_flat):
                x_next = x_next_flat.reshape(self.x.shape)
                f_next = self.eval_f(x_next)
                return (x_next - x_current - (dt / 2) * (f_current + f_next)).flatten()

            def J_trapezoidal_implicit_function(x_next_flat):
                x_next = x_next_flat.reshape(self.x.shape)
                return np.eye(x_next.size) - dt/2 * self.FiniteDifference_JacobianMatrix(x_next)

            x_next_flat_guess = x_current.flatten()
            x_next_flat, converged = self.HybridSolver(x_next_flat_guess, trapezoidal_implicit_function, J_trapezoidal_implicit_function, convergence_threshold, max_iters_newton)

            if not converged:
                print(f"Newton-GCR method failed to converge at time step {n + 1}. Decreasing dt.")
                dt = max(initial_dt, dt / 1.1)
                continue

            self.x = x_next_flat.reshape(self.x.shape)
            y[n + 1] = self.x.copy()

            # Calculate time remaining and adjust dt
            time_remaining = t_stop - self.t
            dt = min(dt, time_remaining)  # Ensure dt does not exceed time remaining

            self.t += dt
            t[n + 1] = self.t
            n += 1

            # Adjust dt based on solution change and phase of simulation
            change = np.linalg.norm(self.x - x_current, np.inf)
            gradual_reduction_threshold = 0.025 * (t_stop - t_start)  

            if self.t >= t_stop - gradual_reduction_threshold:
                dt = max(initial_dt, dt / 1.1)  
            elif change < tolerance / 2 and dt < max_dt:
                dt = min(dt * 1.1, max_dt, time_remaining)  
            #elif change > tolerance:
                #dt = max(initial_dt, dt / 1.5)  # Decrease dt if change is large

            print(f"Time step {n + 1}: dt = {dt}, change = {change}")

        # Trim the arrays to the actual number of steps taken
        t = t[:n + 1]
        y = y[:n + 1]

        return t, y

    # Wrapped functions
    def wrapped_eval_f(self, x_flat):
        x_reshaped = x_flat.reshape((self.N, self.num_psis))
        f_val = self.eval_f(x_reshaped)
        return f_val.flatten()

    def wrapped_J(self, x_flat):
        x_reshaped = x_flat.reshape((self.N, self.num_psis))
        J = self.FiniteDifference_JacobianMatrix(x_reshaped)
        return J

    def backward_euler(self, tspan, dt):
        t_start, t_stop = tspan
        num_steps = int((t_stop - t_start) / dt)
        t = np.zeros(num_steps + 1)
        y = np.zeros((num_steps + 1,) + self.x.shape)
        y[0] = self.x

        self.t = t_start
        t[0] = self.t

        epsilon_f = 1e-1  # Tolerance for function convergence
        epsilon_deltax = 1e-1  # Tolerance for change in x
        epsilon_xrel = np.inf  # Relative tolerance for x
        max_iters = 100  # Maximum iterations for Newton's method

        for n in range(num_steps):
            # run active step for dead tracker and sources
            _ = self.eval_f(active_step=True, dt=dt)
            x_next_flat_guess = self.x.flatten()

            def implicit_function(x_next_flat):
                x_next = x_next_flat.reshape(self.x.shape)
                return (x_next - self.x - dt * self.eval_f(x_next)).flatten()
            
            def J_backward_implicit_function(x_next_flat):
                x_next = x_next_flat.reshape(self.x.shape)
                return np.eye(x_next.size) - dt* self.FiniteDifference_JacobianMatrix(x_next) 

            # Extract only the first element of the tuple (the state x) from the solver
            x_next_flat, converged = self.NewtonNd(x_next_flat_guess, implicit_function,
                                                   custom_J = J_backward_implicit_function)

            self.x = x_next_flat.reshape(self.x.shape)
            print(f"Time step {n + 1}: Sum of x = {np.sum(self.x)}")
            y[n + 1] = self.x
            self.t += dt
            t[n + 1] = self.t
            if self.t % 10 == 0:
                print("Time: ", self.t)

        return t, y
    

    def backward_euler_f(self, tspan, dt):
        t_start, t_stop = tspan
        num_steps = int((t_stop - t_start) / dt)
        t = np.zeros(num_steps + 1)
        y = np.zeros((num_steps + 1,) + self.x.shape)
        y[0] = self.x

        self.t = t_start
        t[0] = self.t

        for n in range(num_steps):
            # run active step for dead tracker and sources
            _ = self.eval_f(active_step=True, dt=dt)
            x_next_flat_guess = self.x.flatten()

            def implicit_function(x_next_flat):
                x_next = x_next_flat.reshape(self.x.shape)
                return (x_next - self.x - dt * self.eval_f(x_next)).flatten()

            # Solve using fsolve
            x_next_flat = fsolve(implicit_function, x_next_flat_guess)

            self.x = x_next_flat.reshape(self.x.shape)
            print(f"Time step {n + 1}: Sum of x = {np.sum(self.x)}")
            y[n + 1] = self.x
            self.t += dt
            t[n + 1] = self.t
            if self.t % 10 == 0:
                print("Time: ", self.t)

        return t, y

    def trapezoidal_method_f(self, tspan, dt):
        t_start, t_stop = tspan
        num_steps = int((t_stop - t_start) / dt)
        t = np.zeros(num_steps + 1)
        y = np.zeros((num_steps + 1,) + self.x.shape)
        y[0] = self.x

        self.t = t_start
        t[0] = self.t

        for n in range(num_steps):
            x_current = self.x
            f_current = self.eval_f(x_current, active_step=True, dt=dt)

            def trapezoidal_implicit_function(x_next_flat):
                x_next = x_next_flat.reshape(self.x.shape)
                f_next = self.eval_f(x_next)
                res = x_next - x_current - (dt / 2) * (f_current + f_next)
                return res.flatten()

            x_next_flat_guess = x_current.flatten()
            x_next_flat = fsolve(trapezoidal_implicit_function, x_next_flat_guess)
            print(f"Time step {n + 1}: before reshaping  = {x_next_flat}")
            self.x = x_next_flat.reshape(self.x.shape)
            y[n + 1] = self.x

            self.t += dt
            t[n + 1] = self.t
            if self.t % 10 == 0:
                print("Time: ", self.t)

        return t, y


In [68]:
import dash
from dash import dcc, html, callback_context
from dash.exceptions import PreventUpdate
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go
import numpy as np
import dash_bootstrap_components as dbc
import base64
import io
import pandas as pd


# Initialize parameters and settings
np.random.seed(42)

css_style = """
body {
    font-family: Arial, sans-serif !important;
}
"""

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.CYBORG, css_style])

# this is the flag for aborting the simulation
is_simulation_aborted = False

@app.callback(
    Output('abort-flag-store', 'data'),
    [Input('abort-button', 'n_clicks')],
    prevent_initial_call=True
)
def abort_simulation(n_clicks):
    return {'is_aborted': True}


location_data = []

@app.callback(
    [Output('locations-table', 'children'),
     Output('status-message', 'children')],
    [Input('add-location-btn', 'n_clicks'),
     Input('clear-locations-btn', 'n_clicks'),
     Input('clear-selected-btn', 'n_clicks')],
    [State('input-location-name', 'value'),
     State('input-x-coordinate', 'value'),
     State('input-y-coordinate', 'value'),
     State('input-population', 'value'),
     State('input-infection-source', 'value'),
     State('input-vaccination-rate', 'value'),
     State('input-mobility-toggle', 'value')],  
    prevent_initial_call=True
)
def update_location_table_and_message(add_clicks, clear_clicks, clear_selected_clicks, name, x, y, population, infection_source, vaccination_rate, mobility):
    global location_data
    triggered_id = callback_context.triggered[0]['prop_id'].split('.')[0]
    message = ""
    
    if triggered_id == 'add-location-btn':
        if name and x and y and population is not None and infection_source is not None and vaccination_rate is not None:
            location_data.append({
                'name': name,
                'x': float(x),
                'y': float(y),
                'population': int(population),
                'infection_source': float(infection_source),
                'vaccination_rate': float(vaccination_rate), 
                 'mobility': "Yes" if mobility else "No"
            })
            message = f"Location {name} added successfully!"
        else:
            message = "All fields are required."
            raise PreventUpdate
    elif triggered_id == 'clear-locations-btn':
        location_data.clear()
        message = "All locations cleared successfully!"
    elif triggered_id == 'clear-selected-btn':
        original_len = len(location_data)
        location_data = [loc for loc in location_data if loc['name'] != name]
        if len(location_data) < original_len:
            message = f"Location {name} removed successfully!"
        else:
            message = f"Location {name} not found."
    
    table_header = [
        html.Thead(html.Tr([
            html.Th("Name"),
            html.Th("X Coordinate"),
            html.Th("Y Coordinate"),
            html.Th("Population"),
            html.Th("Infection Source"),
            html.Th("Vaccination Rate"), 
            html.Th("Mobility")
        ]))
    ]
    table_body = [html.Tbody([
        html.Tr([
            html.Td(loc['name']),
            html.Td(loc['x']),
            html.Td(loc['y']),
            html.Td(loc['population']),
            html.Td("{:.2f}".format(loc['infection_source'])),
            html.Td("{:.2f}".format(loc['vaccination_rate'])), 
            html.Td(loc['mobility']) 
        ]) for loc in location_data
    ])]
    
    return table_header + table_body, message



@app.callback(
    Output('simulation-plot', 'figure'),
    [Input('run-button', 'n_clicks')],
    [State('abort-flag-store', 'data'), 
     State('alpha', 'value'), State('beta', 'value'),
     State('kappa', 'value'),
     State('duration-slider', 'value'), State('time-step-slider', 'value')]  
)
def update_simulation(n_clicks, abort_flag_data, alpha, beta, kappa, duration, time_step):
    global location_data
    p = {}
    
    if n_clicks is None:
        raise PreventUpdate
    
    # Check if the simulation was aborted
    if abort_flag_data.get('is_aborted', False):
        return go.Figure(layout={'mapbox': {'style': "carto-darkmatter", 'center': {'lat': 37.04358, 'lon': -97.23412}, 'zoom': 3}, 'margin': {'l': 0, 't': 0, 'b': 0, 'r': 0}})
    
    original_coords = np.array([
    [29.7604, -95.3698],  # Houston, Texas
    [33.4484, -112.0740],  # Phoenix, Arizona
    [39.7392, -104.9903],  # Denver, Colorado
    [32.7157, -117.1611],  # San Diego, California
    [37.7749, -122.4194],  # San Francisco, California
    [47.6062, -122.3321],  # Seattle, Washington
    [32.7767, -96.7970],  # Dallas, Texas
    [39.9526, -75.1652],  # Philadelphia, Pennsylvania
    [42.3601, -71.0589],  # Boston, Massachusetts
    [38.9072, -77.0369],  # Washington D.C.
    [25.7617, -80.1918],  # Miami, Florida
    [35.2271, -80.8431],  # Charlotte, North Carolina
    [36.1627, -86.7816],  # Nashville, Tennessee
    [45.5152, -122.6784],  # Portland, Oregon
    [29.9511, -90.0715],  # New Orleans, Louisiana
    [44.9778, -93.2650],  # Minneapolis, Minnesota
    [27.9506, -82.4572],  # Tampa, Florida
])
    
    additional_coords = np.array([[loc['x'], loc['y']] for loc in location_data])
    updated_coords = additional_coords if additional_coords.size > 0 else original_coords
    new_N = updated_coords.shape[0]

    populations = [loc['population'] for loc in location_data]
    infection_sources = [loc['infection_source'] for loc in location_data]
    vaccination_rates = [loc['vaccination_rate'] for loc in location_data]
    p['mode'] = 'exp'
    p['coords'] = updated_coords
    p['N'] = new_N
    p['P'] = np.ones([new_N, new_N])
    
    mobility_matrix = []
    for loc in location_data:
        mobility_values = [0.7, 0.3, 1] if loc.get('mobility', 'No') == 'Yes' else [0, 0, 0]
        mobility_matrix.append(mobility_values)
    if mobility_matrix:  
        p['M'] = np.array(mobility_matrix)
    else: 
        p['M'] = np.tile(np.array([0.7, 0.3, 1])[np.newaxis, :], [new_N, 1])
        
    p['alpha'] = np.full(new_N, float(alpha))
    p['beta'] = np.full(new_N, float(beta))
    p['kappa'] = np.full(new_N, float(kappa))
    p['mu'] = np.tile(0.00001 * np.array([1.00, 1.00, 1.00])[np.newaxis, :], [new_N, 1])
    p['gamma'] = np.full(new_N, 0.1)
    p['zeta'] = np.full(new_N, 0.1)
    p['u0'] = np.full(new_N, 0)
    p['q0'] = np.array([10] + [0]*(new_N-1)) if not infection_sources else np.array(infection_sources)
    print(p['u0'])
    p['v'] = np.array([0]*new_N) if not vaccination_rates else np.array(vaccination_rates)
    print(p['v'])
    p['S0'] = np.random.randint(10000, 30000, size=new_N) if not populations else np.array(populations)
    print(p['S0'])


    sim = Simulator(p)

    default_duration = 350
    default_time_step = 0.5
    duration = duration if duration is not None else default_duration
    time_step = time_step if time_step is not None else default_time_step

    if time_step == 0:
        time_step = 1e-6
    tspan = (0, duration)
    dt = time_step

    timesteps, state_trajectories, tau_values = sim.forward_euler(tspan, dt, return_tau=True)

    # print(state_trajectories[-1])

    fig = go.Figure()

    fig.update_layout(
        mapbox_style="carto-darkmatter",
        mapbox=dict(center=dict(lat=37.04358, lon=-97.23412), zoom=2),
        margin={'l': 0, 't': 0, 'b': 0, 'r': 0},
        updatemenus=[{
            'type': 'buttons',
            'buttons': [
                {
                    'label': 'Play',
                    'method': 'animate',
                    'args': [None, {'frame': {'duration': 100, 'redraw': True}, 'fromcurrent': True}],
                },
                {
                    'label': 'Pause',
                    'method': 'animate',
                    'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate'}],
                }
            ],
            'direction': 'left',
            'pad': {'r': 10, 't': 87},
            'showactive': False,
            'x': 0.1,
            'xanchor': 'right',
            'y': 0,
            'yanchor': 'top'
        }]
    )

    frames = []

    for idx, (t, state, tau) in enumerate(zip(timesteps, state_trajectories, tau_values)):
        if idx % 10 != 0:
            continue

        S = state[:, 0]
        I = state[:, 1]
        R = state[:, 2]
        Total = S + I + R
        S_range = S.max() - S.min() if S.max() - S.min() != 0 else np.inf
        I_range = I.max() - I.min() if I.max() - I.min() != 0 else np.inf
        R_range = R.max() - R.min() if R.max() - R.min() != 0 else np.inf
        T_range = Total.max() - Total.min() if Total.max() - Total.min() != 0 else np.inf

        S_normalized = (S - S.min()) / S_range
        I_normalized = (I - I.min()) / I_range
        R_normalized = (R - R.min()) / R_range
        # T_normalized = 0.5 + (Total - Total.min()) / (2 * T_range)
        T_normalized = Total/Total.max()

        S_scale = S / Total
        I_scale = I / Total
        R_scale = R / Total

        max_size = 50
        S_sizes = T_normalized * max_size * S_scale
        I_sizes = T_normalized * max_size * I_scale + S_sizes
        R_sizes = T_normalized * max_size * R_scale + I_sizes


        frame_traces = []


        frame_traces.append(go.Scattermapbox(
            lat=p['coords'][:, 0],
            lon=p['coords'][:, 1],
            mode='markers',
            marker=dict(size=[s+2 for s in R_sizes], color='black'),  # Adjust the size as needed for border width
            name='Recovered_border',
            showlegend=False
        ))

        frame_traces.append(go.Scattermapbox(
            lat=p['coords'][:, 0],
            lon=p['coords'][:, 1],
            mode='markers',
            marker=dict(size=R_sizes, color='green', opacity=1),
            name='Recovered',
            hoverinfo='text',
            hovertext=['Recovered: ' + '{:0.0f}'.format(r) for r in R]
        ))

        frame_traces.append(go.Scattermapbox(
            lat=p['coords'][:, 0],
            lon=p['coords'][:, 1],
            mode='markers',
            marker=dict(size=[s+2 for s in I_sizes], color='black'),  
            name='I_border',
            showlegend=False
        ))

        frame_traces.append(go.Scattermapbox(
            lat=p['coords'][:, 0],
            lon=p['coords'][:, 1],
            mode='markers',
            marker=dict(size=I_sizes, color='#E62020', opacity=1),
            name='Infected',
            hoverinfo='text',
            hovertext=['Infected: ' + '{:0.0f}'.format(inff) for inff in I]
        ))

        frame_traces.append(go.Scattermapbox(
            lat=p['coords'][:, 0],
            lon=p['coords'][:, 1],
            mode='markers',
            marker=dict(size=[s+2 for s in S_sizes], color='black'),  
            name='Sus_border',
            showlegend=False
        ))

        frame_traces.append(go.Scattermapbox(
            lat=p['coords'][:, 0],
            lon=p['coords'][:, 1],
            mode='markers',
            marker=dict(size=S_sizes, color='blue', opacity=1),
            name='Susceptible',
            hoverinfo='text',
            hovertext=['Susceptible: ' + '{:0.0f}'.format(s) for s in S]
        ))

        for i in range(p['N']):
            for j in range(i+1, p['N']):
                if np.sum(tau[i, j, :]) > 0:
                    frame_traces.append(go.Scattermapbox(
                        lat=[p['coords'][i, 0], p['coords'][j, 0]],
                        lon=[p['coords'][i, 1], p['coords'][j, 1]],
                        mode='lines',
                        line=dict(width=min(2 * np.sum(tau[i, j, :]),5), color='yellow'),
                        hoverinfo='none',
                        showlegend=False
                    ))

        frames.append(go.Frame(data=frame_traces, name=str(idx)))

    slider_steps = []
    for idx, frame in enumerate(frames):
        slider_step = {
            "args": [
                [frame.name],
                {"frame": {"duration": 100, "redraw": True}, "mode": "immediate", "transition": {"duration": 100}}
            ],
            "label": f"{int(dt*float(frame.name))}",
            "method": "animate"
        }
        slider_steps.append(slider_step)

    fig.update_layout(
        sliders=[{
            "active": 0,
            "yanchor": "top",
            "xanchor": "left",
            "currentvalue": {
                "font": {"size": 16, "color": "white"},
                "prefix": "Time: ",
                "visible": True,
                "xanchor": "right"
            },
            "transition": {"duration": 300, "easing": "cubic-in-out"},
            "pad": {"b": 10, "t": 10},
            "len": 0.9,
            "x": 0.1,
            "y": 0,
            "steps": slider_steps,
            "bgcolor": "#000000",  # Black background for slider
            "bordercolor": "#666666",
            "borderwidth": 1,
            "font": {"color": "white"}
        }],
        mapbox_style="carto-darkmatter",
        mapbox=dict(center=dict(lat=37.04358, lon=-97.23412), zoom=3),
        updatemenus=[{
            'type': 'buttons',
            'buttons': [
                {
                    'label': 'Play',
                    'method': 'animate',
                    'args': [None, {'frame': {'duration': 100, 'redraw': True}, 'fromcurrent': True}],
                },
                {
                    'label': 'Pause',
                    'method': 'animate',
                    'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate'}],
                }
            ],
            'direction': 'left',
            'pad': {'r': 0, 't': 87},
            'showactive': False,
            'x': 0.1,
            'xanchor': 'right',
            'y': 0,
            'yanchor': 'top',
            'bgcolor': '#000000'  # Black background for buttons
        }],
        margin={'l': 0, 't': 0, 'b': 0, 'r': 0},
        legend=dict(x=0, y=1, orientation='h', bgcolor='rgba(0,0,0,0.7)'),
        font=dict(family="Arial, sans-serif", size=12, color="white"),
        paper_bgcolor='black',  
        plot_bgcolor='black'  
        )
    fig.frames = frames
    fig.add_traces(frames[0].data)

    is_simulation_aborted = False

    return fig

@app.callback(
    Output('duration-value', 'children'),
    [Input('duration-slider', 'value')]
    )
def update_duration_display(value):
        return f"{value} days"

@app.callback(
        Output('time-step-value', 'children'),
        [Input('time-step-slider', 'value')]
    )
def update_time_step_display(value):
        return f"Time step: {value}"
    
@app.callback(
    Output('infection-source-value', 'children'),
    [Input('input-infection-source', 'value')]
)
def update_infection_source_display(value):
    return f"Infection source: {value}"

@app.callback(
    Output('vaccination-rate-value', 'children'),
    [Input('input-vaccination-rate', 'value')]
)
def update_vaccination_rate_display(value):
    return f"Vaccination rate: {value:.2f}"
    
@app.callback(
    [Output('alpha', 'value'), Output('beta', 'value'),
     Output('kappa', 'value'),
     Output('alpha-value', 'children'), Output('beta-value', 'children'),
     Output('kappa-value', 'children')],
    [Input('preset-1', 'n_clicks'), Input('preset-2', 'n_clicks'),
     Input('alpha', 'value'), Input('beta', 'value'),
     Input('kappa', 'value')],
    prevent_initial_call=True
)
def update_values_and_displays(preset1, preset2, alpha, beta, kappa):
    ctx = callback_context
    if not ctx.triggered:
        triggered_id = 'No clicks yet'
    else:
        triggered_id = ctx.triggered[0]['prop_id'].split('.')[0]

    if triggered_id == 'preset-1':
        return 0.35, 0.05, 0.03, "0.35", "0.05", "0.03"
    elif triggered_id == 'preset-2':
        return 0.45, 0.08, 0.06, "0.45", "0.08", "0.06"
    else:
        return alpha, beta, kappa, f"{alpha:.2f}", f"{beta:.2f}", f"{kappa:.2f}"
        
app.layout = dbc.Container(
    [
        dcc.Store(id='abort-flag-store', data={'is_aborted': False}),
        dcc.Store(id='location-data-store', storage_type='session', data=[]),
        dbc.Row(
            dbc.Col(
                html.H4(
                    "Simufection (Computational Engineering & Data Analytics)",
                    className="text-white bg-primary p-2 mb-2 text-center"
                ),
                width=12
            )
        ),
        dbc.Row(
            [
                dbc.Col(
                    [
                        dbc.Card(
                            [
                                html.H5("Input Parameters", className="card-title"),
                                html.Div(
                                    [
                                        html.Label("α (Infection Rate)", className="mb-1"),
                                        dcc.Slider(
                                            id='alpha',
                                            min=0,
                                            max=1,
                                            step=0.01,
                                            value=0.42,
                                             marks={0: '0', 0.25: '0.25', 0.5: '0.5', 0.75: '0.75', 1: '1'},
                                        ),
                                        html.Div(id='alpha-value', children="α: 0.42"),
                                    ],
                                    className="mb-4"
                                ),
                                html.Div(
                                    [
                                        html.Label("β (Recovery Rate)", className="mb-1"),
                                        dcc.Slider(
                                            id='beta',
                                            min=0,
                                            max=1,
                                            step=0.01,
                                            value=0.07,
                                             marks={0: '0', 0.25: '0.25', 0.5: '0.5', 0.75: '0.75', 1: '1'},
                                        ),
                                        html.Div(id='beta-value', children="β: 0.07"),
                                    ],
                                    className="mb-4"
                                ),
                                html.Div(
                                    [
                                        html.Label("κ (Mortality Rate)", className="mb-1"),
                                        dcc.Slider(
                                            id='kappa',
                                            min=0,
                                            max=1,
                                            step=0.01,
                                            value=0.05,
                                             marks={0: '0', 0.25: '0.25', 0.5: '0.5', 0.75: '0.75', 1: '1'},
                                        ),
                                        html.Div(id='kappa-value', children="κ: 0.05"),
                                    ],
                                    className="mb-4"
                                ),
                            ],
                            body=True,
                        ),
                        dbc.Card(
                            [
                                html.H5("Dynamic Parameters", className="card-title"),
                                html.Div(
                                    [
                                        html.Label("Duration (days)", className="mb-1"),
                                        dcc.Slider(
                                            id='duration-slider',
                                            min=0,
                                            max=2000,
                                            step=1,
                                            value=350,
                                            marks={i: str(i) for i in range(0, 2001, 500)},
                                        ),
                                        html.Div(id='duration-value', children="Duration: 350 days"),
                                    ],
                                    className="mb-4"
                                ),
                                html.Div(
                                    [
                                        html.Label("Time Step", className="mb-1"),
                                        dcc.Slider(
                                            id='time-step-slider',
                                            min=0,
                                            max=1,
                                            step=0.001,
                                            value=0.5,
                                             marks={0: '0', 0.25: '0.25', 0.5: '0.5', 0.75: '0.75', 1: '1'},
                                        ),
                                        html.Div(id='time-step-value', children="Time step: 0.5"),
                                    ],
                                    className="mb-4"
                                ),
                            ],
                            body=True,
                            className="mt-4"
                        ),
                    ],
                    width=3
                ),
                dbc.Col(
                    [
                        dbc.Card(
                            [
                                html.H5("Locations", className="card-title"),
                                dbc.InputGroup(
                                    [
                                        dbc.InputGroupText("Name"),
                                        dbc.Input(id="input-location-name", placeholder="Enter a name", type="text"),
                                    ],
                                    className="mb-3"
                                ),
                                dbc.InputGroup(
                                    [
                                        dbc.InputGroupText("X"),
                                        dbc.Input(id="input-x-coordinate", placeholder="Enter X-coords", type="number"),
                                    ],
                                    className="mb-3"
                                ),
                                dbc.InputGroup(
                                    [
                                        dbc.InputGroupText("Y"),
                                        dbc.Input(id="input-y-coordinate", placeholder="Enter Y-coords", type="number"),
                                    ],
                                    className="mb-3"
                                ),
                                dbc.InputGroup(
                                    [
                                        dbc.InputGroupText("Population"),
                                        dbc.Input(id="input-population", placeholder="Population", type="number"),
                                    ],
                                    className="mb-3"
                                ),
                                dbc.Row(
                                    [
                                        dbc.Col(dbc.Switch(id="input-mobility-toggle", label="Mobility", value=False), width=10),
                                    ],
                                    className="mb-3"
                                ),
                                html.Div(
                                    [
                                        html.Label("Infection Source", className="mb-1"),
                                        dcc.Slider(
                                            id='input-infection-source',
                                            min=0,
                                            max=100,
                                            step=1,
                                            value=50,
                                            marks={i: str(i) for i in range(0, 101, 25)},
                                        ),
                                        html.Div(id='infection-source-value', children="Infection source: 50"),
                                    ],
                                    className="mb-4"
                                ),
                                html.Div(
                                    [
                                        html.Label("Vaccination Rate", className="mb-1"),
                                        dcc.Slider(
                                            id='input-vaccination-rate',
                                            min=0,
                                            max=1,
                                            step=0.01,
                                            value=0.5,
                                             marks={0: '0', 0.25: '0.25', 0.5: '0.5', 0.75: '0.75', 1: '1'},
                                        ),
                                        html.Div(id='vaccination-rate-value', children="Vaccination rate: 0.5"),
                                    ],
                                    className="mb-4"
                                ),
                                dbc.Row(
                                    [
                                        dbc.Col(dcc.Upload(
                                            id='upload-data',
                                            children=html.Button('Upload Data', id='upload-button', className='btn btn-info', n_clicks=0),
                                            multiple=True
                                        ), width=12),
                                    ],
                                    className="mb-4"
                                ),
                                dbc.Row(
                                    [
                                        dbc.Col(html.Button('Add Location', id='add-location-btn', className='btn btn-primary', n_clicks=0), width=4),
                                        dbc.Col(html.Button('Clear Selected', id='clear-selected-btn', className='btn btn-warning', n_clicks=0), width=4),
                                        dbc.Col(html.Button('Clear All', id='clear-locations-btn', className='btn btn-danger', n_clicks=0), width=4),
                                    ],
                                    className="mb-3"
                                ),
                                html.Div(id='status-message', className="text-center"),
                            ],
                            body=True
                        ),
                    ],
                    width=3
                ),
                dbc.Col(
                    [
                        dbc.Tabs(
                            [
                                dbc.Tab(label='Map', children=[
                                    dcc.Loading(
                                        id="loading-map",
                                        type="cube",
                                        children=dcc.Graph(id='simulation-plot')
                                    )
                                ], tab_id='tab-map'),
                                dbc.Tab(label='Locations of Interest', children=[
                                    html.Div(id='locations-content', children=[
                                        dbc.Table(id='locations-table', bordered=True, dark=True, hover=True, responsive=True, striped=True),
                                    ])
                                ], tab_id='tab-locations'),
                                dbc.Tab(label='Statistics', children=[
                                    html.Div(id='stats-content', children=["Statistics content will be added here."])
                                ], tab_id='tab-stats'),
                            ],
                            id='tabs',
                            active_tab='tab-map'
                        ),
                        dbc.Card(
                            dbc.CardBody(
                                [
                                    dbc.Row(
                                        [
                                            dbc.Col(html.Button('Run Sim', id='run-button', n_clicks=0, className='btn btn-primary'), width=3),
                                            dbc.Col(html.Button('Abort Sim', id='abort-button', n_clicks=0, className='btn btn-danger'), width=3),
                                            dbc.Col(html.Button('Preset 1', id='preset-1', n_clicks=0, className='btn btn-dark'), width=3),
                                            dbc.Col(html.Button('Preset 2', id='preset-2', n_clicks=0, className='btn btn-dark'), width=3),
                                        ],
                                        className='mt-4'
                                    ),
                                ]
                            ),
                            className='mt-4'
                        ),
                    ],
                    md=6
                ),
            ]
        ),
    ],
    fluid=True,
    className="dbc"
)

if __name__ == '__main__':
    app.run_server(debug=True)




[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[25795 10860 15390 21964 21284 16265 26850 14426 24423 21363 26023 18322
 11685 10769 12433 15311 15051]
Shape of q: (17,)
Shape of u: (17,)
Shape of x[:,0]: (17,)
Shape of q: (17,)
Shape of u: (17,)
Shape of x[:,0]: (17,)
Shape of q: (17,)
Shape of u: (17,)
Shape of x[:,0]: (17,)
Shape of q: (17,)
Shape of u: (17,)
Shape of x[:,0]: (17,)
Shape of q: (17,)
Shape of u: (17,)
Shape of x[:,0]: (17,)
Shape of q: (17,)
Shape of u: (17,)
Shape of x[:,0]: (17,)
Shape of q: (17,)
Shape of u: (17,)
Shape of x[:,0]: (17,)
Shape of q: (17,)
Shape of u: (17,)
Shape of x[:,0]: (17,)
Shape of q: (17,)
Shape of u: (17,)
Shape of x[:,0]: (17,)
Shape of q: (17,)
Shape of u: (17,)
Shape of x[:,0]: (17,)
Shape of q: (17,)
Shape of u: (17,)
Shape of x[:,0]: (17,)
Shape of q: (17,)
Shape of u: (17,)
Shape of x[:,0]: (17,)
Shape of q: (17,)
Shape of u: (17,)
Shape of x[:,0]: (17,)
Shape of q: (17,)
Shape of u: (17,)
Shape of x[:,0]: (17

In [3]:
pip install dash dash-bootstrap-components plotly numpy scipy scikit-learn

  Using cached dash_bootstrap_components-1.5.0-py3-none-any.whl.metadata (5.2 kB)
Using cached dash_bootstrap_components-1.5.0-py3-none-any.whl (221 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [57]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go
import numpy as np
import dash_bootstrap_components as dbc
import json

# Initialize parameters and settings
np.random.seed(42)

css_style = """
body {
    font-family: Arial, sans-serif !important;
}
"""

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.CYBORG, css_style])

@app.callback(
    [Output('preset_id', 'data')],
    [Input('preset-1', 'n_clicks'), Input('preset-2', 'n_clicks'), Input('preset-3', 'n_clicks')]
)
def update_presets(preset1_clicks, preset2_clicks, preset3_clicks):
    ctx = dash.callback_context
    if not ctx.triggered:
        return [[0]]
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]
        if button_id == 'preset-1':
            return [[1]]
        elif button_id == 'preset-2':
            return [[2]]
        elif button_id == 'preset-3':
            return [[3]]

@app.callback(
    Output('simulation-plot', 'figure'),
    [Input('run-button', 'n_clicks'), Input('preset_id', 'data')],
    [State('preset_id', 'data')]
)
def update_simulation(n_clicks,_, preset_id):
    if n_clicks is None:
        raise dash.exceptions.PreventUpdate

    # alpha_array = np.array([float(a) for a in alpha.split(',')])
    # beta_array = np.array([float(b) for b in beta.split(',')])
    # kappa_array = np.array([float(k) for k in kappa.split(',')])
    # v_array = np.array([float(vv) for vv in v.split(',')])

    # p['alpha'] = alpha_array
    # p['beta'] = beta_array
    # p['kappa'] = kappa_array
    # p['v'] = v_array
    if preset_id == None:
        p = p0
    else:
        if preset_id[0] == 0:
            p = p0
        if preset_id[0] == 1:
            p = p1
        if preset_id[0] == 2:
            p = p2
        if preset_id[0] == 3:
            p = p3


    # Assume the Simulator class and forward_euler method are defined elsewhere
    sim = Simulator(p)
    tspan = (0, 200)
    dt = 0.1
    timesteps, state_trajectories, tau_values = sim.forward_euler(tspan, dt, return_tau=True)
    fig = go.Figure()
    fig.update_layout(
        mapbox_style="carto-darkmatter",
        mapbox=dict(center=dict(lat=37.04358, lon=-97.23412), zoom=2),
        margin={'l': 0, 't': 0, 'b': 0, 'r': 0},
        updatemenus=[{
            'type': 'buttons',
            'buttons': [
                {
                    'label': 'Play',
                    'method': 'animate',
                    'args': [None, {'frame': {'duration': 100, 'redraw': True}, 'fromcurrent': True}],
                },
                {
                    'label': 'Pause',
                    'method': 'animate',
                    'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate'}],
                }
            ],
            'direction': 'left',
            'pad': {'r': 10, 't': 87},
            'showactive': False,
            'x': 0.1,
            'xanchor': 'right',
            'y': 0,
            'yanchor': 'top'
        }]
    )

    frames = []

    for idx, (t, state, tau) in enumerate(zip(timesteps, state_trajectories, tau_values)):
        if idx % 10 != 0:
            continue

        S = state[:, 0]
        I = state[:, 1]
        R = state[:, 2]
        Total = S + I + R
        S_range = S.max() - S.min() if S.max() - S.min() != 0 else np.inf
        I_range = I.max() - I.min() if I.max() - I.min() != 0 else np.inf
        R_range = R.max() - R.min() if R.max() - R.min() != 0 else np.inf
        T_range = Total.max() - Total.min() if Total.max() - Total.min() != 0 else np.inf

        S_normalized = (S - S.min()) / S_range
        I_normalized = (I - I.min()) / I_range
        R_normalized = (R - R.min()) / R_range
        # T_normalized = 0.5 + (Total - Total.min()) / (2 * T_range)
        T_normalized = Total/Total.max()

        S_scale = S / Total
        I_scale = I / Total
        R_scale = R / Total

        max_size = 50
        S_sizes = T_normalized * max_size * S_scale
        I_sizes = T_normalized * max_size * I_scale + S_sizes
        R_sizes = T_normalized * max_size * R_scale + I_sizes


        frame_traces = []


        frame_traces.append(go.Scattermapbox(
            lat=p['coords'][:, 0],
            lon=p['coords'][:, 1],
            mode='markers',
            marker=dict(size=[s+2 for s in R_sizes], color='black'),  # Adjust the size as needed for border width
            name='Recovered_border',
            showlegend=False
        ))

        frame_traces.append(go.Scattermapbox(
            lat=p['coords'][:, 0],
            lon=p['coords'][:, 1],
            mode='markers',
            marker=dict(size=R_sizes, color='green', opacity=1),
            name='Recovered',
            hoverinfo='text',
            hovertext=['Recovered: ' + '{:0.0f}'.format(r) for r in R]
        ))

        frame_traces.append(go.Scattermapbox(
            lat=p['coords'][:, 0],
            lon=p['coords'][:, 1],
            mode='markers',
            marker=dict(size=[s+2 for s in I_sizes], color='black'),  # Adjust the size as needed for border width
            name='I_border',
            showlegend=False
        ))

        frame_traces.append(go.Scattermapbox(
            lat=p['coords'][:, 0],
            lon=p['coords'][:, 1],
            mode='markers',
            marker=dict(size=I_sizes, color='#E62020', opacity=1),
            name='Infected',
            hoverinfo='text',
            hovertext=['Infected: ' + '{:0.0f}'.format(inff) for inff in I]
        ))

        frame_traces.append(go.Scattermapbox(
            lat=p['coords'][:, 0],
            lon=p['coords'][:, 1],
            mode='markers',
            marker=dict(size=[s+2 for s in S_sizes], color='black'),  # Adjust the size as needed for border width
            name='Sus_border',
            showlegend=False
        ))

        frame_traces.append(go.Scattermapbox(
            lat=p['coords'][:, 0],
            lon=p['coords'][:, 1],
            mode='markers',
            marker=dict(size=S_sizes, color='orange', opacity=1),
            name='Susceptible',
            hoverinfo='text',
            hovertext=['Susceptible: ' + '{:0.0f}'.format(s) for s in S]
        ))

        for i in range(p['N']):
            for j in range(i+1, p['N']):
                if np.sum(tau[i, j, :]) > 0:
                    frame_traces.append(go.Scattermapbox(
                        lat=[p['coords'][i, 0], p['coords'][j, 0]],
                        lon=[p['coords'][i, 1], p['coords'][j, 1]],
                        mode='lines',
                        line=dict(width=min(2 * np.sum(tau[i, j, :]),5), color='yellow'),
                        hoverinfo='none',
                        showlegend=False
                    ))

        frames.append(go.Frame(data=frame_traces, name=str(idx)))

    slider_steps = []
    for idx, frame in enumerate(frames):
        slider_step = {
            "args": [
                [frame.name],
                {"frame": {"duration": 100, "redraw": True}, "mode": "immediate", "transition": {"duration": 100}}
            ],
            "label": f"{int(dt*float(frame.name))}",
            "method": "animate"
        }
        slider_steps.append(slider_step)

    fig.update_layout(
        sliders=[{
            "active": 0,
            "yanchor": "top",
            "xanchor": "left",
            "currentvalue": {
                "font": {"size": 16, "color": "white"},
                "prefix": "Time: ",
                "visible": True,
                "xanchor": "right"
            },
            "transition": {"duration": 300, "easing": "cubic-in-out"},
            "pad": {"b": 10, "t": 10},
            "len": 0.9,
            "x": 0.1,
            "y": 0,
            "steps": slider_steps,
            "bgcolor": "#000000",  # Black background for slider
            "bordercolor": "#666666",
            "borderwidth": 1,
            "font": {"color": "white"}
        }],
        mapbox_style="carto-darkmatter",
        mapbox=dict(center=dict(lat=37.04358, lon=-97.23412), zoom=3),
        updatemenus=[{
            'type': 'buttons',
            'buttons': [
                {
                    'label': 'Play',
                    'method': 'animate',
                    'args': [None, {'frame': {'duration': 100, 'redraw': True}, 'fromcurrent': True}],
                },
                {
                    'label': 'Pause',
                    'method': 'animate',
                    'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate'}],
                }
            ],
            'direction': 'left',
            'pad': {'r': 0, 't': 87},
            'showactive': False,
            'x': 0.1,
            'xanchor': 'right',
            'y': 0,
            'yanchor': 'top',
            'bgcolor': '#000000'  # Black background for buttons
        }],
        margin={'l': 0, 't': 0, 'b': 0, 'r': 0},
        legend=dict(x=0, y=1, orientation='h', bgcolor='rgba(0,0,0,0.7)'),
        font=dict(family="Arial, sans-serif", size=12, color="white"),
        paper_bgcolor='black',  # Set paper background color to black
        plot_bgcolor='black'  # Set plot background color to black
        )
    fig.frames = frames
    fig.add_traces(frames[0].data)
    return fig


app.layout = dbc.Container(
    [
        
        dbc.Row(
            dbc.Col(
                html.H4("Simufection (6.7300)", className="text-black p-2 mb-2 text-center", style={'backgroundColor': 'rgba(255, 255, 255, 0.8)'}),
                width=12
            )
        ),
        dbc.Row(
            [
                dbc.Col(
                    [
                        # dbc.Row([
                        #     dbc.Col(html.Label("α", className="mr-2"), width=3),
                        #     dbc.Col(dcc.Input(id='alpha', type='text', value=','.join(['0.42'] * p['N']), className='form-control'), width=10)
                        # ], className="mb-3"),
                        # dbc.Row([
                        #     dbc.Col(html.Label("β", className="mr-2"), width=3),
                        #     dbc.Col(dcc.Input(id='beta', type='text', value=','.join(['0.07'] * p['N']), className='form-control'), width=10)
                        # ], className="mb-3"),
                        # dbc.Row([
                        #     dbc.Col(html.Label("κ", className="mr-2"), width=3),
                        #     dbc.Col(dcc.Input(id='kappa', type='text', value=','.join(['0.05'] * p['N']), className='form-control'), width=10)
                        # ], className="mb-3"),
                        # dbc.Row([
                        #     dbc.Col(html.Label("ν", className="mr-2"), width=3),
                        #     dbc.Col(dcc.Input(id='v', type='text', value=','.join(['0.2'] * p['N']), className='form-control'), width=10)
                        # ], className="mb-3"),
                        dcc.Store(id='preset_id', data=[0]),
                        html.Button('Run Simulator', id='run-button', n_clicks=1, className='btn btn-warning btn-block my-2'),
                        html.Button('High Infection Case', id='preset-1', n_clicks=0, className='btn btn-info btn-block my-2'),
                        html.Button('Vaccination Case', id='preset-2', n_clicks=0, className='btn btn-info btn-block my-2'),
                        html.Button('Lockdown Case', id='preset-3', n_clicks=0, className='btn btn-info btn-block my-2')
                    ],
                    width=2
                ),
                dbc.Col(
                    dcc.Loading(
                        id="loading-1",
                        type="cube",
                        children=dcc.Graph(id='simulation-plot')
                    ),
                    width=10
                )
            ]
        )
    ],
    fluid=True
)

if __name__ == '__main__':
    app.run_server(debug=True)

us_coords = np.array([
    [40.6892, -74.0445],  # Statue of Liberty, New York
    [31.9686, -99.9018], # Texas
    [28.3852, -81.5639],  # Walt Disney World Resort, Orlando, Florida
    [38.8977, -77.0365],  # The White House, Washington D.C.
    [36.1147, -115.1728], # Las Vegas Strip, Las Vegas, Nevada
    [43.8791, -103.4591], # Mount Rushmore, South Dakota
    [37.8267, -122.4233], # Alcatraz Island, San Francisco, California
    [29.9584, -90.0644]   # French Quarter, New Orleans, Louisiana
])
p0 = {}
p0['coords'] = np.array(us_coords)
p0['N'] = us_coords.shape[0]
p0['P'] = np.ones([p0['N'], p0['N']])
p0['M'] = np.tile(np.array([0.7, 0.3, 1])[np.newaxis, :], [p0['N'], 1])
p0['S0'] = np.random.randint(10000, 20000, size=p0['N'])
p0['mu'] = np.tile(0.00001 * np.array([1.00, 1.00, 1.00])[np.newaxis, :], [p0['N'], 1])
p0['gamma'] = np.repeat(0.2, p0['N'])
p0['zeta'] = np.repeat(0.2, p0['N'])
p0['u0'] = np.repeat(10, p0['N'])
p0['q0'] = np.array([10] + [0]*7)
p0['mode'] = 'exp'
p0['alpha'] = np.repeat(0.31, p0['N'])
p0['beta'] = np.repeat(0.01, p0['N'])
p0['kappa'] = np.repeat(0.00, p0['N'])
p0['v'] = np.array(8*[0])

Shape of q: (8,)
Shape of u: (8,)
Shape of x[:,0]: (8,)
Shape of q: (8,)
Shape of u: (8,)
Shape of x[:,0]: (8,)
Shape of q: (8,)
Shape of u: (8,)
Shape of x[:,0]: (8,)
Shape of q: (8,)
Shape of u: (8,)
Shape of x[:,0]: (8,)
Shape of q: (8,)
Shape of u: (8,)
Shape of x[:,0]: (8,)
Shape of q: (8,)
Shape of u: (8,)
Shape of x[:,0]: (8,)
Shape of q: (8,)
Shape of u: (8,)
Shape of x[:,0]: (8,)
Shape of q: (8,)
Shape of u: (8,)
Shape of x[:,0]: (8,)
Shape of q: (8,)
Shape of u: (8,)
Shape of x[:,0]: (8,)
Shape of q: (8,)
Shape of u: (8,)
Shape of x[:,0]: (8,)
Shape of q: (8,)
Shape of u: (8,)
Shape of x[:,0]: (8,)
Shape of q: (8,)
Shape of u: (8,)
Shape of x[:,0]: (8,)
Shape of q: (8,)
Shape of u: (8,)
Shape of x[:,0]: (8,)
Shape of q: (8,)
Shape of u: (8,)
Shape of x[:,0]: (8,)
Shape of q: (8,)
Shape of u: (8,)
Shape of x[:,0]: (8,)
Shape of q: (8,)
Shape of u: (8,)
Shape of x[:,0]: (8,)
Shape of q: (8,)
Shape of u: (8,)
Shape of x[:,0]: (8,)
Shape of q: (8,)
Shape of u: (8,)
Shape of x[:,0